# GOAL OF LABORATORY WORK
Using the trapezoidal rule of definite integral to approximate the regions or
compartmentalized sections under a graph of a given function and also calculating the area as
well.

# TASK DEFINITION
The specific problem that is solved in the laboratory work (1 paragraph).
Calculating the execution time of a serial program, that is, the estimated time required for calculating the values of different upper and lower bounds of an integral, which is given as follows:


Choosing an arbitrary value of epsilon, with a maximum value of 10^-7. Convert the serial program written to a parallel program using OpenMP by using reduction concept and
synchronization methods, such as; lock, atomic and critical sections. Later, count the speedup of executions based on different thread numbers.

# BRIEF THEORY
OpenMP (Open Multi-Processing) is an application programming interface (API) that
supports multi-platform shared memory multiprocessing programming in C, C++, and Fortran on most platforms, instruction set architectures and operating systems, including Solaris, AIX, HP-UX, Linux, macOS, and Windows. It consists of a set of compiler directives, library routines, and environment variables that influence run-time behaviorAn application built with the hybrid model of parallel programming can run on a computer cluster using both OpenMP and Message Passing Interface (MPI), such that OpenMP is used for parallelism within a (multi-core) node while MPI is used for parallelism between nodes. There have also been efforts to run OpenMP on software distributed shared memory systems, to translate OpenMP into MPI and to extend OpenMP for non-shared memory systems.
![img](images/CropperScreenshot_2019-06-21_21:19:42.png)
Synchronization Methods in OpenMP.
To control issues pertaining to race conditions, synchronization methods are used to
protect data conflicts.

- Reduction – reduction clause: reduction(op:list). A local copy of each list of variable is made and initialized depending on the op. Example; 1 for ‘*’, the sign is assigned to number 1 as its identity. Updates occur on the local copy, then reduced into a single value and combined with the original global value. The construct for reduction is combined with the for construct: #pragma omp parallel for reduction(+:plus)
- Critical – allows on thread of code execute at a time. If a thread is performing a computation and hits a critical section in a block which isn’t free, it has to wait for its turn. When a thread finishes its computation, it releases for the next thread in queue. The construct for critical section is #pragma omp critical; and it consumes all executions do by individual threads every time, i.e. on every overhead.
- Lock – is the lowest level of mutual exclusion synchronization; Lock Routines: omp_init_lock(), omp_set_lock(), omp_unset_lock(), omp_destroy_lock(), and omp_test_lock. Setting locks on essential section of codes in parallel for a thread at a time. Memory is released for the next thread when it has completed it task. Sometimes, with the help of omp_test_lock(), it queries to find free threads.
- Atomic – allows for quick updates of values in memory. It applies to a simple binary operation when updating a value. Example of its use is during an increment, decrement or performing operations such as read or write. The construct for atomic section is #pragma omp atomic. 

## Trapezoidal Rule
The trapezoidal rule is a technique for approximating the definite integral by approximating the region under the graph of the function as a trapezoid and calculating its area. Most often used in numerical analysis.

![img](images/CropperScreenshot_2019-06-21_21:19:51.png)

# ALGORITHM (METHOD) OF IMPLEMENTATION

In [28]:
%cat int.c

#include <omp.h>

#include <stdio.h>
#include <stdlib.h> 


// #include <boost/math/quadrature/trapezoidal.hpp>

#define A ((double)_A)
#define B ((double)_B)
#define N ((double)_N)
#define PREC ((double)_PREC)


#ifndef INTEG
# define INTEG integrate0
#endif


#ifndef COMPUTE_SUM
# define COMPUTE_SUM compute_sum_reduction
#endif

#include <math.h>




double f(double x) {
	double a = (1./x) * sin(1./x);
	return pow(a, 2);
}

double f_int(double a, double b) {
	return (1./4.) * (  2. * (b-a)/(a*b) + sin(2/b) - sin(2/a) );
}

double F(double x) {
	return -1/(2*x) + (1/4) * sin(2/x);
}



double compute_sum_raw(size_t p, double h, double a){
	double sum = 0;

	// #pragma omp parallel for
	for(size_t j = 1; j < p; j += 1)
	{
		double y = f(a + j*h);
		sum += y;
	}
	return sum;
}

double compute_sum_atomic(size_t p, double h, double a){
	double sum = 0;

	#pragma omp parallel for
	for(size_t j = 1; j < p; j += 1)
	{
		double y = f(a + j*h);
		#pragma omp atomic
		sum += y;
	}
	return sum;


# RESULT AND EXPERIMENTS

Task:
    
1. Choose precision ε
2. Calculate integral with different A and B values from the table
3. Calculate execution time of serial program
4. Write a parallel program with:
    1. atomic
    2. Critical sections
    3. Locks
    4. reduction
5. Count speedup with different thread number
6. Fill the table (for each point of 4a-4d)


In [1]:
import subprocess
import os


def compile(*defs, **defskw):
    args = [f"-D{k}" for k in defs] + [f"-D{k}={v}" for k, v in defskw.items()]
    _cmd = 'g++ int.c -o int -fopenmp -lm -I boost_1_70_0/ -std=c++14'.split() + args
    # print(' '.join(_cmd))
    cmd = subprocess.run(_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  
    if(cmd.stdout): print('cmd.stdout', cmd.stdout)
    if(cmd.stderr): print('cmd.stderr', cmd.stderr)
    
def run(env=None):
    cmd = subprocess.run('./int', stdout=subprocess.PIPE, stderr=subprocess.PIPE, env=env)
    return cmd.stdout.decode('utf8')
    


## Reduce  configuraions

Lets test different reduce configuratios.

In [2]:
env = os.environ.copy()
env['OMP_NUM_THREADS'] = str(4)

for x in ['raw', 'atomic', 'locks', 'reduction', 'critical']:
    print(f'Executing integration with {x} locking')
    compile(_A=1, _B=10, _PREC=-6.30E-11, COMPUTE_SUM=f'compute_sum_{x}')

    %timeit run(env)
    print()



Executing integration with raw locking
cmd.stderr b"clang: warning: treating 'c' input as 'c++' when in C++ mode, this behavior is deprecated [-Wdeprecated]\nclang: error: unsupported option '-fopenmp'\nclang: error: unsupported option '-fopenmp'\n"


OSError: [Errno 8] Exec format error: './int'

We clearly see the advatange of using OpenMP reductions..

## Thread speedup

Lets analyze how the number of threads affects the speed of the execution.

In [3]:
env = os.environ.copy()
env['OMP_NUM_THREADS'] = str(4)


compile(_A=1, _B=10, _PREC=-6.30E-11, COMPUTE_SUM=f'compute_sum_reduction')

print(f'Executing integration with 4 threads')
%timeit run(env)

env = os.environ.copy()
env['OMP_NUM_THREADS'] = str(2)


compile(_A=1, _B=10, _PREC=-6.30E-11, COMPUTE_SUM=f'compute_sum_reduction')

print(f'Executing integration with 2 threads')
%timeit run(env)

env = os.environ.copy()
env['OMP_NUM_THREADS'] = str(1)

print(f'Executing integration with 1 threads (!!)')
compile(_A=1, _B=10, _PREC=-6.30E-11, COMPUTE_SUM=f'compute_sum_reduction')

%timeit run(env)

cmd.stderr b"clang: warning: treating 'c' input as 'c++' when in C++ mode, this behavior is deprecated [-Wdeprecated]\nclang: error: unsupported option '-fopenmp'\nclang: error: unsupported option '-fopenmp'\n"
Executing integration with 4 threads


OSError: [Errno 8] Exec format error: './int'

More threads equal to a faster execution

### Alternative integration implementaion

The program offered several ways to impove beyond the ordinary sum. An alternative version was also developed. In order to increase performance a combination of OpenMP task and OpenMP normal multithreading was used. No because of the design no reductions were possible.

In [4]:
for ts in [2, 4,8, 16, 32]:
    env = os.environ.copy()
    env['OMP_NUM_THREADS'] = str(ts)
    
    compile(_A=1, _B=10, _PREC=-6.30E-11, INTEG='integrate1')
    
    print(f"Alternative tasks based integration with {ts} threas")
    %timeit run(env)
    
    compile(_A=1, _B=10, _PREC=-6.30E-11, INTEG='integrate0')
    
    print(f"Default tasks based integration with {ts} threas")
    %timeit run(env)
    print()

cmd.stderr b"clang: warning: treating 'c' input as 'c++' when in C++ mode, this behavior is deprecated [-Wdeprecated]\nclang: error: unsupported option '-fopenmp'\nclang: error: unsupported option '-fopenmp'\n"
Alternative tasks based integration with 2 threas


OSError: [Errno 8] Exec format error: './int'

We clearly see the advantage of the task based design which maximize resource utilization without compromizing flexiblity.

# Filling the data

Lets collect the data required by the task

In [5]:
xs = [ 
    ( 0.00001 , 0.0001 , -2.77e-11),
    (  0.0001 ,  0.001 ,   1.9e-10), 
    (   0.001 ,   0.01 ,  2.05E-11),
    (    0.01 ,    0.1 , -2.22E-12),
    (     0.1 ,      1 ,  8.67E-11),
    (       1 ,     10 , -6.00E-11),
    (      10 ,    100 , -6.30E-11)
]
for a, b, eps in xs:
    compile(_A=a, _B=b, _PREC=eps, INTEG='integrate1')
    print(f"Intergating on ({a}, {b}) interval with {eps} precision")
    print("Time taken:", end=' ')
    %timeit run(env)
    print(run().split('\n')[0])
    print()

cmd.stderr b"clang: warning: treating 'c' input as 'c++' when in C++ mode, this behavior is deprecated [-Wdeprecated]\nclang: error: unsupported option '-fopenmp'\nclang: error: unsupported option '-fopenmp'\n"
Intergating on (1e-05, 0.0001) interval with -2.77e-11 precision
Time taken: 

OSError: [Errno 8] Exec format error: './int'

# CONCLUSION
The three method (locks, atomics, critical sections) shows less performance because of spending
a lot of time on synchronizing and blocks, while the operation of summing is very expensive. So
most of time threads just waiting for another threads unlock (or store the new value in memory
or send the sync signals). The reduction shows good boost up.